This notebook explains you how to use ADOPT without MSAs and what information can be extracted from the tool.

# Execution

Firstly, we include the packages which ADOPT needs to calculate Z scores.

In [ ]:
from adopt import MultiHead, ZScorePred

Next we need a protein sequence. Here we use the FASTA sequence of cellular tumor antigen p53 with UniProtKB entry P04637 as an example. We also give it a characteristic name.

In [ ]:
SEQUENCE="MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD"
BRMID="p53"

Next, we have to choose a model and a training and test dataset. All models are based on Facebook’s Evolutionary Scale Modeling (ESM) library. Three libraries are available: ESM-1b, ESM-1v and ESM-MSA. For more details, see [here](https://github.com/facebookresearch/esm). Additionally, there is a combined version. If you want to use the ESM with Multiple Seuence Alignment (MSA), you can use a docker container provided by Peptone. If you do not need the MSA, you can simply specify the library.
        
You can also choose the training and test dataset. We recommend you that you use the Transformer which was trained on the cleared dataset from 1325 proteins of the CheZoD database and tested on 117 proteins from the CheZoD database.

In [ ]:
MODEL_TYPE = "esm-1b"
STRATEGY = "train_on_cleared_1325_test_on_117_residue_split"

Now it is time to set up the multi-head level representation of the Transformer. The first time you do this it takes several minutes because the library has to be downloaded. Each following setup will be much faster (usually a few seconds).

In [ ]:
multi_head = MultiHead(MODEL_TYPE, SEQUENCE, BRMID)
representation, tokens = multi_head.get_representation()

Now let us calculate the Z scores with ADOPT.

In [ ]:
z_score_pred = ZScorePred(STRATEGY, MODEL_TYPE)
predicted_z_scores = z_score_pred.get_z_score(representation)

# Visualisation

Now, let us have a look at the predicted Z scores.

In [ ]:
import numpy as np
# Residue number starts at 1, but array at 0
resnr=np.arange(1, 1+len(predicted_z_scores))
import plotly.express as px
fig = px.line(x=resnr, y=predicted_z_scores, title="", labels=dict(x="Residue number", y="Z score"))
fig.show()

The Z scores are connected to secondary chemical shifts and describe the local disorder of a residue within a sequence. Residues with Z>8 are part of strutured regions and residues with Z<8 are part of unstrutured regions. 

We can see that p53 has a 100 residue long, unstructured N-terminal and a short unstructured C-terminal region. Furthermore, there is an unstructured region around residue 300 while most of the rest of the structure is folded. ADOPT also identifies the turn between different strands of beta sheets which have Z score around 9.

Now let's divide the residues in four categories: Fully disordered (FDIS), partially disordered (PDIS), structured (STRUCT) and flexible loops (FLEX) based on the predicted Z score. 

In [ ]:
from adopt import utils, constants
from spacy import displacy
sum_list=[]
for (item1, item2) in zip(list(SEQUENCE), list(resnr)):
    sum_list.append(item1+str(item2))
residues_state = utils.get_residue_class(predicted_z_scores)
residues_state_and_nr=[{'label':residues_state[i]['label'], 'start':sum(len(x) for x in sum_list[:i]), 'end':sum(len(x) for x in sum_list[:i+1])} for i in range(len(sum_list))]
ex = [{"text": ''.join(sum_list),
       "ents": residues_state_and_nr,
       "title": None}]

options = {"ents": constants.res_colors.keys(), "colors": constants.res_colors}
html = displacy.render(ex, style="ent", manual=True, options=options, jupyter=True)

# Interpretation

By dividing residues into four categories of flexibility, ADOPT predicts the known structurally distinct regions found within the tumour suppressor gene p53. In its functional state, p53 is a homotetrameric protein capable of binding both DNA and other proteins. This functionality is enabled by the presence of 5 domains located throughout 3 broader regions. These regions can be divided coarsely into the flexible amino terminal (aa 1 - 100), structured core domain (aa 101 - 300) and flexible carboxyl domain (aa 301 - 393). We can visualise the gradual transition from the more flexible N-terminal region to ordered core domain with the transition from PDIS (partially disordered) labelled (aa P98 - Q100) to FLEX (flexible) labelled (aa K101 - Y103) followed by more consistently STRUCT (structured) labelled residues. Similarly, the transition back to regions of greater flexibility between the core and carboxyl domains can be seen in the PDIS labelled residues (K292 - P295) followed by FDIS (fully disordered) labelled H296, H297 etc. This transition point perfectly aligns with the end of the DNA binding domain located mainly in the core region (aa 94 - 292) (Chillemi, G. et al 2016, https://doi.org/10.1101/cshperspect.a028308). 

ADOPT is able to successfully capture the functional disorder and structure found throughout p53’s carboxyl domain. As previously mentioned, p53 forms a homotetramer to bind specific protein targets with higher efficacy. This is enabled by the presence of a tetramerization domain, consisting of a beta sheet and an alpha helix between residues 326 to 356 (Chène, P. 2001, https://doi.org/10.1038/sj.onc.1204373). The results from ADOPT outline a stretch of FDIS and PDIS labelled (aa P295 - L323) linking the end of the core domain to FLEX labelled residues (D324 - E326), demarcating the start of this binding domain where STRUCT labeled residues can be found (Y327 - L350). Following the tetramerization domain, a transition back into the intrinsically disordered C-terminal domain can be seen in PDIS labelled (aa D351 - G356) followed by primarily FDIS labelled residues until the C-terminus. This region has been dubbed a “chameleon sequence” as its intrinsic flexibility enables promiscuous binding with a number of interaction partners through conformational flexibility. Interestingly, computational analysis of this region in response to different hydrophobic conditions identifies a subregion (residues 381 to 386) which forms a partial helix (Kumar, A. et al 2020, https://doi.org/10.1016/j.abb.2020.108342). ADOPT is able to partially capture this in the labelling of PDIS residues M384, F385 and T387 amongst primarily FDIS labeled residues. These results indicate ADOPT’s ability to accurately predict biologically relevant structural order and disorder within the context of the tumour suppressor gene p53.